In [363]:
# import 
import pandas as pd
import numpy as np
import folium
import json

In [364]:
# 스타벅스 매장리스트
sbuck_df = pd.read_excel('./busan_starbucks.xlsx', header=0)
sbuck_df.head()

,매장명,위도,경도,매장타입,주소
0,금정남산동DT,35.262466,129.091353,generalDT,부산광역시 금정구 중앙대로 1989 (남산동)
1,금정구청DT,35.238784,129.092964,generalDT,부산광역시 금정구 중앙대로 1730 (부곡동)
2,온천장역DT,35.224006,129.086560,generalDT,부산광역시 금정구 식물원로 13 (장전동)
3,부산대정문,35.232107,129.084565,general,부산광역시 금정구 부산대학로64번길 12 (장전동)
4,부산대역,35.230722,129.088165,general,"부산광역시 금정구 장전로12번길 64, 1~4층 (장전동)"


In [365]:
sbuck_df['주소'][136].split()[1]

'해운대구'

In [372]:
gu_lsit = []
for item in sbuck_df['주소']:
    gu = item.split()[1]
    gu_lsit.append(gu)

sbuck_df['구'] = gu_lsit
sbuck_df.head(100)

,매장명,위도,경도,매장타입,주소,구
0,금정남산동DT,35.262466,129.091353,generalDT,부산광역시 금정구 중앙대로 1989 (남산동),금정구
1,금정구청DT,35.238784,129.092964,generalDT,부산광역시 금정구 중앙대로 1730 (부곡동),금정구
2,온천장역DT,35.224006,129.086560,generalDT,부산광역시 금정구 식물원로 13 (장전동),금정구
3,부산대정문,35.232107,129.084565,general,부산광역시 금정구 부산대학로64번길 12 (장전동),금정구
4,부산대역,35.230722,129.088165,general,"부산광역시 금정구 장전로12번길 64, 1~4층 (장전동)",금정구
...,...,...,...,...,...,...
95,부산안락DT,35.195859,129.106954,generalDT,부산광역시 동래구 충렬대로 446 (안락동),동래구
96,동래미남DT,35.207244,129.071966,generalDT,부산광역시 동래구 충렬대로 101 (온천동),동래구
97,부산사직,35.198093,129.060473,general,부산광역시 동래구 사직북로 24 (사직동),동래구
98,부산온천장역,35.221082,129.085506,general,"부산광역시 동래구 중앙대로 1523, SK허브스카이위아아파트 1층 (온천동)",동래구


#### 부산광역시 행정구역별 위경도 전처리

- 대한민국 행정구역 위경도엑셀파일에서 부산만 필터링

In [366]:
busan_loc_df = pd.read_excel('./대한민국_행정구역_위경도.xlsx')
busan_loc_df

,docity,do,city,longitude,latitude
0,NaN,강원,강릉시,128.878497,37.749136
1,NaN,강원,고성군,128.470164,38.377961
2,NaN,강원,동해시,129.116633,37.521931
3,NaN,강원,삼척시,129.167489,37.447086
4,NaN,강원,속초시,128.594167,38.204275
...,...,...,...,...,...
290,NaN,충청,충주시,127.928144,36.988181
291,NaN,충청,태안군,126.299975,36.742667
292,NaN,충청,한누리대로,127.289926,36.485450
293,NaN,충청,홍성군,126.662908,36.598361


In [367]:
busan_gu_df = busan_loc_df[busan_loc_df['do'] == '부산시']
busan_gu_df.tail()

,docity,do,city,longitude,latitude
133,NaN,부산시,연제구,129.082075,35.173186
134,NaN,부산시,영도구,129.070186,35.088117
135,NaN,부산시,중구,129.034508,35.103217
136,NaN,부산시,해운대구,129.165808,35.160019
137,NaN,부산시,기장군,129.222287,35.244775


In [368]:
busan_gu_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16 entries, 122 to 137
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   docity     0 non-null      float64
 1   do         16 non-null     object 
 2   city       16 non-null     object 
 3   longitude  16 non-null     float64
 4   latitude   16 non-null     float64
dtypes: float64(3), object(2)
memory usage: 768.0+ bytes


In [369]:
# seoul_gu_df 수정
busan_gu_df = busan_gu_df[['do', 'city', 'latitude', 'longitude']]
columns = {'do':'도', 'city':'구', 'latitude':'위도','longitude':'경도'}
busan_gu_df.rename(columns=columns, inplace=True)
busan_gu_df.reset_index(drop=True, inplace=True)

In [370]:
busan_gu_df

,도,구,위도,경도
0,부산시,강서구,35.209164,128.982908
1,부산시,금정구,35.240078,129.094319
2,부산시,남구,35.133408,129.086500
3,부산시,동구,35.135894,129.059175
4,부산시,동래구,35.201872,129.085856
5,부산시,부산진구,35.159953,129.055319
6,부산시,북구,35.194181,128.992475
7,부산시,사상구,35.149467,128.993333
8,부산시,사하구,35.101428,128.977042
9,부산시,서구,35.094836,129.026378


In [301]:
busan_gu_df.to_excel('./busan_gu_info.xlsx', index=False)

##### 부산시 구별 스타벅스 매장수 구하기

In [373]:
sbuck_gu_counts = sbuck_df.pivot_table(index='구', 
                     values='매장명',
                     aggfunc='count').rename(columns={'매장명': '스타벅스_매장수'})

In [340]:
sbuck_gu_counts.head()

,스타벅스_매장수
구,
강서구,5
금정구,8
기장군,6
남구,6
동구,4


In [374]:
busan_gu_df = busan_gu_df[['도', '구', '경도', '위도']]

In [375]:
busan_gu_df

,도,구,경도,위도
0,부산시,강서구,128.982908,35.209164
1,부산시,금정구,129.094319,35.240078
2,부산시,남구,129.086500,35.133408
3,부산시,동구,129.059175,35.135894
4,부산시,동래구,129.085856,35.201872
5,부산시,부산진구,129.055319,35.159953
6,부산시,북구,128.992475,35.194181
7,부산시,사상구,128.993333,35.149467
8,부산시,사하구,128.977042,35.101428
9,부산시,서구,129.026378,35.094836


In [376]:
busan_sgg = pd.merge(busan_gu_df, sbuck_gu_counts, how='left', on='구')
busan_sgg

,도,구,경도,위도,스타벅스_매장수
0,부산시,강서구,128.982908,35.209164,5
1,부산시,금정구,129.094319,35.240078,8
2,부산시,남구,129.086500,35.133408,6
3,부산시,동구,129.059175,35.135894,4
4,부산시,동래구,129.085856,35.201872,9
5,부산시,부산진구,129.055319,35.159953,19
6,부산시,북구,128.992475,35.194181,5
7,부산시,사상구,128.993333,35.149467,7
8,부산시,사하구,128.977042,35.101428,7
9,부산시,서구,129.026378,35.094836,4


In [377]:
busan_pop_df = pd.read_excel('./부산_주민등록인구_202305.xlsx')
busan_pop_df

c:\Program Files\Python311\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,행정구역(시군구)별,2023.05
0,부산광역시,1562595
1,중구,24066
2,서구,53593
3,동구,46152
4,영도구,54248
5,부산진구,180263
6,동래구,120032
7,남구,118686
8,북구,123510
9,해운대구,170221


In [378]:
busan_pop_df.drop([0], axis=0, inplace=True)
busan_pop_df

,행정구역(시군구)별,2023.05
1,중구,24066
2,서구,53593
3,동구,46152
4,영도구,54248
5,부산진구,180263
6,동래구,120032
7,남구,118686
8,북구,123510
9,해운대구,170221
10,사하구,141510


In [379]:
busan_pop_df = busan_pop_df[['행정구역(시군구)별', '2023.05']]
columns = {'행정구역(시군구)별': '구', '2023.05' : '계'}
busan_pop_df.rename(columns=columns, inplace=True)
busan_pop_df.reset_index(drop=True, inplace=True)
busan_pop_df

,구,계
0,중구,24066
1,서구,53593
2,동구,46152
3,영도구,54248
4,부산진구,180263
5,동래구,120032
6,남구,118686
7,북구,123510
8,해운대구,170221
9,사하구,141510


In [381]:
busan_pop_df.rename(columns={'구':'구', '계':'인구수'}, inplace=True)
busan_pop_df

,구,인구수
0,중구,24066
1,서구,53593
2,동구,46152
3,영도구,54248
4,부산진구,180263
5,동래구,120032
6,남구,118686
7,북구,123510
8,해운대구,170221
9,사하구,141510


In [382]:
busan_pop_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   구       16 non-null     object
 1   인구수     16 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 388.0+ bytes


In [391]:
busan_sgg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   도         16 non-null     object 
 1   구         16 non-null     object 
 2   경도        16 non-null     float64
 3   위도        16 non-null     float64
 4   스타벅스_매장수  16 non-null     int64  
dtypes: float64(2), int64(1), object(2)
memory usage: 772.0+ bytes


In [390]:
busan_sgg.merge(busan_pop_df, how='outer', on='구')

,도,구,경도,위도,스타벅스_매장수,인구수
0,부산시,강서구,128.982908,35.209164,5.0,NaN
1,부산시,금정구,129.094319,35.240078,8.0,NaN
2,부산시,남구,129.086500,35.133408,6.0,NaN
3,부산시,동구,129.059175,35.135894,4.0,NaN
4,부산시,동래구,129.085856,35.201872,9.0,NaN
5,부산시,부산진구,129.055319,35.159953,19.0,NaN
6,부산시,북구,128.992475,35.194181,5.0,NaN
7,부산시,사상구,128.993333,35.149467,7.0,NaN
8,부산시,사하구,128.977042,35.101428,7.0,NaN
9,부산시,서구,129.026378,35.094836,4.0,NaN


In [314]:
busan_sgg = busan_sgg[['도', '구', '경도', '위도', '스타벅스_매장수', '계']]
busan_sgg.tail()

,도,구,경도,위도,스타벅스_매장수,계
11,부산시,연제구,129.082075,35.173186,9,NaN
12,부산시,영도구,129.070186,35.088117,2,NaN
13,부산시,중구,129.034508,35.103217,6,NaN
14,부산시,해운대구,129.165808,35.160019,32,NaN
15,부산시,기장군,129.222287,35.244775,6,NaN


In [315]:
# 사업체수
busan_biz = pd.read_excel('./busan_biz.xlsx')
busan_biz.tail()

,구,사업체수,종사자수
11,강서구,30038,143600
12,연제구,23946,96109
13,수영구,19613,62351
14,사상구,36104,124257
15,기장군,17930,77117


In [316]:
busan_sgg_stat = pd.merge(busan_sgg, busan_biz, how='left', on='구') 
busan_sgg_stat.tail()

,도,구,경도,위도,스타벅스_매장수,계,사업체수,종사자수
11,부산시,연제구,129.082075,35.173186,9,NaN,23946,96109
12,부산시,영도구,129.070186,35.088117,2,NaN,11616,46978
13,부산시,중구,129.034508,35.103217,6,NaN,18306,69033
14,부산시,해운대구,129.165808,35.160019,32,NaN,40946,154115
15,부산시,기장군,129.222287,35.244775,6,NaN,17930,77117


In [317]:
columns = {'도':'도', '구':'구','경도':'경도','위도':'위도','스타벅스_매장수':'스타벅스_매장수','계_x':'인구수','사업체수':'사업체수','종사자수계':'종사자수'}

busan_sgg_stat.rename(columns=columns,inplace=True)

In [318]:
busan_sgg_stat.to_excel('./busan_sgg_stat.xlsx', index=False)
busan_sgg_stat

,도,구,경도,위도,스타벅스_매장수,계,사업체수,종사자수
0,부산시,강서구,128.982908,35.209164,5,NaN,30038,143600
1,부산시,금정구,129.094319,35.240078,8,NaN,26714,102359
2,부산시,남구,129.086500,35.133408,6,NaN,24190,93744
3,부산시,동구,129.059175,35.135894,4,NaN,18208,76827
4,부산시,동래구,129.085856,35.201872,9,NaN,27667,97700
5,부산시,부산진구,129.055319,35.159953,19,NaN,44355,174317
6,부산시,북구,128.992475,35.194181,5,NaN,21049,66682
7,부산시,사상구,128.993333,35.149467,7,NaN,36104,124257
8,부산시,사하구,128.977042,35.101428,7,NaN,29196,113587
9,부산시,서구,129.026378,35.094836,4,NaN,11376,45728


##### 데이터 시각화

In [319]:
sbuck_df.tail()

,매장명,위도,경도,매장타입,주소,구
132,해운대 엑스더스카이,35.159965,129.169756,general,부산광역시 해운대구 달맞이길30,해운대구
133,센텀신세계3F,35.169063,129.129066,general,부산광역시 해운대구 센텀남대로 35 (우동),해운대구
134,해운대중동역,35.167185,129.167886,general,부산광역시 해운대구 좌동순환로 6 (중동),해운대구
135,센텀몰1F,35.169810,129.127783,general,부산광역시 해운대구 센텀4로 15 (우동),해운대구
136,그랜드조선 부산,35.160034,129.163051,general,부산광역시 해운대구 해운대해변로 292 (중동),해운대구


In [320]:
for idx in sbuck_df:
    print(idx)

매장명
위도
경도
매장타입
주소
구


In [329]:
!pip install folium

In [330]:
sbuck_map = folium.Map(location=[35.1795543, 129.0756416], tiles='Stamen Terrain', zoom_start=11)

for idx in sbuck_df.index:
    lat = sbuck_df.loc[idx, '위도']
    lng = sbuck_df.loc[idx, '경도']
    name = sbuck_df.loc[idx, '매장명']

    folium.CircleMarker(location=[lat,lng], fill=True, fill_color='green', fill_opacity=1, popup=name,
                        color='yellow', weight=1, radius=3).add_to(sbuck_map)
    
sbuck_map

In [323]:
sbuck_map2 = folium.Map(location=[35.1795543, 129.0756416], tiles='Stamen Terrain', zoom_start=11)

for idx in sbuck_df.index:
    lat = sbuck_df.loc[idx, '위도']
    lng = sbuck_df.loc[idx, '경도']
    name = sbuck_df.loc[idx, '매장명']
    type = sbuck_df.loc[idx, '매장타입']

    # 매장타입별 색상 
    fillcolor = ''
    size = 2
    if type == 'general':
        fillcolor = 'gray'
        size = 1
    elif type == 'reserve':
        fillcolor = 'balck'
        size = 5
    elif type == 'generalDT':
        fillcolor = 'red'
        size = 3
    elif type == 'generalWT':
        fillcolor = 'blue'
        size = 3
    
    folium.CircleMarker(location=[lat,lng], fill=True, fill_color=fillcolor, fill_opacity=1, popup=name,
                        color='yellow', weight=1, radius=size).add_to(sbuck_map2)
    
sbuck_map2

In [324]:
# 시군구경계표시
sgg_geojson_path = './hangjeongdong_부산광역시.geojson'
busan_sgg_geo = json.load(open(sgg_geojson_path, encoding = 'utf-8'))

sbuck_map3 = folium.Map(location=[35.1795543, 129.0756416], tiles='CartoDB dark_matter', zoom_start=11)

folium.GeoJson(busan_sgg_geo, name='Busan행정구역').add_to(sbuck_map3)
sbuck_map3

In [325]:
sbuck_mean = busan_sgg_stat['스타벅스_매장수'].mean()
sbuck_mean

8.5625

In [326]:
# 시군별 스타벅스 매장수를 버블지도로 시각화
sgg_geojson_path = './hangjeongdong_부산광역시.geojson'
busan_sgg_geo = json.load(open(sgg_geojson_path, encoding = 'utf-8'))

sbuck_map3 = folium.Map(location=[35.1795543, 129.0756416], tiles='CartoDB dark_matter', zoom_start=11)

folium.GeoJson(busan_sgg_geo, name='Busan행정구역').add_to(sbuck_map3)

for idx in busan_sgg_stat.index:
    lat = busan_sgg_stat.loc[idx, '위도']
    lng = busan_sgg_stat.loc[idx, '경도']
    count = busan_sgg_stat.loc[idx, '스타벅스_매장수']

    if count > sbuck_mean: # 구별 평균치보다 매장수가 많음
        fillcolor = 'red'
    else:
        fillcolor = 'blue'
    
    folium.CircleMarker(location=[lat,lng], color='#ffff00', fill_color=fillcolor, fill_opacity=0.7, weight=1.5, radius=count/2, popup=count).add_to(sbuck_map3)

sbuck_map3

In [327]:
busan_sgg_geo = json.load(open(sgg_geojson_path, encoding='utf-8'))
busan_sgg_geo

{'type': 'FeatureCollection',
 'name': 'temp',
 'crs': {'type': 'name',
  'properties': {'name': 'urn:ogc:def:crs:OGC:1.3:CRS84'}},
 'features': [{'type': 'Feature',
   'properties': {'OBJECTID': 425,
    'adm_nm': '부산광역시 중구 중앙동',
    'adm_cd': '2101051',
    'adm_cd2': '2611051000',
    'sgg': '26110',
    'sido': '26',
    'sidonm': '부산광역시',
    'sggnm': '중구'},
   'geometry': {'type': 'MultiPolygon',
    'coordinates': [[[[129.04001607594367, 35.111424681629],
       [129.0427298111509, 35.110722158835344],
       [129.042347191582, 35.11034344765853],
       [129.0423152220015, 35.11029796763163],
       [129.041990699126, 35.10982871044188],
       [129.0414304709214, 35.109010295030885],
       [129.04088010913745, 35.10811134155702],
       [129.0445525636584, 35.10580584261088],
       [129.04373774886275, 35.10492439441351],
       [129.04036624750276, 35.10703056658817],
       [129.0398182034819, 35.10584181288945],
       [129.0399339281879, 35.10576888949396],
       [129.0

In [328]:
# 시군구경계표시
sgg_geojson_path = './hangjeongdong_부산광역시.geojson'
busan_sgg_geo = json.load(open(sgg_geojson_path, encoding = 'utf-8'))

sbuck_map4 = folium.Map(location=[35.1795543, 129.0756416], tiles='CartoDB dark_matter', zoom_start=11)

folium.GeoJson(busan_sgg_geo, name='Busan행정구역').add_to(sbuck_map4)

folium.Choropleth(
    geo_data=busan_sgg_geo,
    data=busan_sgg_stat,
    columns=['구', '스타벅스_매장수'],
    fill_color= 'Accent', #'YlGn', 'Spectral', 'RdYlGn', 'Accent', 'Reds', 'Greens', 'Blues', 'Oranges', 'PuOr'
    fill_opacity=0.7,
    line_opacity=0.5,
    key_on='properties.name'  # 이게 핵심 - key 값이 없으면 지도에 색상표시 못함
).add_to(sbuck_map4)    

sbuck_map4

ValueError: key_on `'properties.name'` not found in GeoJSON.

In [ ]:
busan_sgg_stat

,도,구,경도,위도,스타벅스_매장수,계,사업체수,종사자수
0,부산시,강서구,128.982908,35.209164,5,NaN,30038,143600
1,부산시,금정구,129.094319,35.240078,8,NaN,26714,102359
2,부산시,남구,129.086500,35.133408,6,NaN,24190,93744
3,부산시,동구,129.059175,35.135894,4,NaN,18208,76827
4,부산시,동래구,129.085856,35.201872,9,NaN,27667,97700
5,부산시,부산진구,129.055319,35.159953,19,NaN,44355,174317
6,부산시,북구,128.992475,35.194181,5,NaN,21049,66682
7,부산시,사상구,128.993333,35.149467,7,NaN,36104,124257
8,부산시,사하구,128.977042,35.101428,7,NaN,29196,113587
9,부산시,서구,129.026378,35.094836,4,NaN,11376,45728


In [ ]:
# 서울시 인구수별 분포
busan_pop_choropleth = folium.Map(location=[35.1795543, 129.0756416], tiles='CartoDB dark_matter', zoom_start=11)

folium.Choropleth(geo_data=busan_sgg_geo, data=busan_sgg_stat,
                  columns=['구', '인구수'], fill_color='Purples', fill_opacity=0.7, line_opacity=0.5,
                  key_on='properties.name').add_to(busan_pop_choropleth)

busan_pop_choropleth

- 결론 : 구에 살고있는 인구수와 스타벅스 매장수는 관계 거의 없다.

In [ ]:
# 서울시 사업체수별 분포
busan_biz_choropleth = folium.Map(location=[35.1795543, 129.0756416], tiles='CartoDB dark_matter', zoom_start=11)

folium.Choropleth(geo_data=busan_sgg_geo, data=busan_sgg_stat,
                  columns=['구', '사업체수'], fill_color='Purples', fill_opacity=0.7, line_opacity=0.5,
                  key_on='properties.name').add_to(busan_biz_choropleth)

busan_biz_choropleth

- 결론 : 각 구별 사업체수와 스타벅스 매장수는 깊은 연관이 있다.

         동작구는 사당역 근처 리저브 매장을 폐점했다.(매출이 부진 예상)